In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, MaxPool2D, Dense, Conv2D, Flatten
from keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')




In [ ]:
data=pd.read_csv('../input/age-gender-and-ethnicity-face-data-csv/age_gender.csv')
data.head()

In [ ]:
def values_to_array(pixels):
    arr=np.array(pixels.split(),'float64')
    arr=arr.reshape(48,48)
    return arr
data['pixels']= data['pixels'].apply(values_to_array)

In [ ]:
train_img, test_img, train_age,test_age= train_test_split(data['pixels'], data['age'], test_size=0.2, random_state=42)

In [ ]:
plt.figure(figsize=(10,8))
sns.distplot(train_age)
plt.title("Age Distribution")
plt.show()

In [ ]:
# Change dimension of Data
def change_image_dimension(data):
    data=np.reshape(data.to_list(), (len(data), 48,48,1))
    return data

In [ ]:
train_img= change_image_dimension(train_img)
test_img= change_image_dimension(test_img)
train_img=train_img/255.0
test_img=test_img/255.0

In [ ]:
# CALLBACK FUNCTIONS
def schedule(epoch,lr):
    if epoch>=5:
        return 0.0001
    return 0.001
early_stop=EarlyStopping(monitor='val_mean_absolute_error',patience=2)
learning_rate_scheduler=LearningRateScheduler(schedule)
reduceLR=ReduceLROnPlateau(monitor='val_mean_absolute_error',patience=1,min_lr=0.00001)

In [ ]:
model=Sequential()
model.add(Conv2D(128,(3,3),activation='relu',input_shape=(48,48,1)))
model.add(MaxPool2D((2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='relu'))

In [ ]:
model.compile(optimizer='adam', loss='mse',metrics=[keras.metrics.mean_absolute_error])
r=model.fit(train_img, train_age, validation_data=(test_img, test_age),epochs=2, callbacks=[reduceLR])

In [ ]:
plt.plot(r.history['loss'])
plt.plot(r.history['val_loss'])


In [ ]:
plt.plot(r.history['mean_absolute_error'])
plt.plot(r.history['val_mean_absolute_error'])
